In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime as dt
import math

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
pd_data_base_path = '/content/drive/MyDrive/Preppin Data/Data/'

In [64]:
f = pd.ExcelFile(pd_data_base_path + 'New Customers.xlsx')
list_of_dfs = []
for sheet in f.sheet_names:
  df = f.parse(sheet)
  df['month'] = sheet
  list_of_dfs.append(df)
customers = pd.concat(list_of_dfs, ignore_index=True)

In [65]:
customers.head()

,ID,Joining Day,Demographic,Value,month,Demographiic,Demagraphic
0,490910,3,Ethnicity,White,January,NaN,NaN
1,490910,3,Date of Birth,5/23/1981,January,NaN,NaN
2,490910,3,Account Type,Basic,January,NaN,NaN
3,369221,18,Ethnicity,Black,January,NaN,NaN
4,369221,18,Date of Birth,3/4/2019,January,NaN,NaN


In [66]:
customers[customers.Demagraphic.notnull()]

,ID,Joining Day,Demographic,Value,month,Demographiic,Demagraphic
2226,182172,12,NaN,Other,October,NaN,Ethnicity
2227,182172,12,NaN,11/11/1940,October,NaN,Date of Birth
2228,182172,12,NaN,Basic,October,NaN,Account Type
2229,293881,29,NaN,Black,October,NaN,Ethnicity
2230,293881,29,NaN,3/10/1963,October,NaN,Date of Birth
...,...,...,...,...,...,...,...
2497,754452,4,NaN,11/13/1955,October,NaN,Date of Birth
2498,754452,4,NaN,Basic,October,NaN,Account Type
2499,111501,12,NaN,Black,October,NaN,Ethnicity
2500,111501,12,NaN,5/3/1999,October,NaN,Date of Birth


In [67]:
customers[customers.Demographiic.notnull()]

,ID,Joining Day,Demographic,Value,month,Demographiic,Demagraphic
1716,107924,2,NaN,Black,August,Ethnicity,NaN
1717,107924,2,NaN,9/2/1975,August,Date of Birth,NaN
1718,107924,2,NaN,Gold,August,Account Type,NaN
1719,534142,26,NaN,Black,August,Ethnicity,NaN
1720,534142,26,NaN,11/9/1954,August,Date of Birth,NaN
...,...,...,...,...,...,...,...
1969,900582,29,NaN,12/9/1963,August,Date of Birth,NaN
1970,900582,29,NaN,Gold,August,Account Type,NaN
1971,333873,16,NaN,Black,August,Ethnicity,NaN
1972,333873,16,NaN,8/31/1994,August,Date of Birth,NaN


In [68]:
customers['Demographic'] = np.where(customers['month']=='October', customers['Demagraphic'], customers['Demographic'])
customers['Demographic'] = np.where(customers['month']=='August', customers['Demographiic'], customers['Demographic'])

In [69]:
customers = customers[['ID','Joining Day','Demographic','Value','month']]
customers.head()

,ID,Joining Day,Demographic,Value,month
0,490910,3,Ethnicity,White,January
1,490910,3,Date of Birth,5/23/1981,January
2,490910,3,Account Type,Basic,January
3,369221,18,Ethnicity,Black,January
4,369221,18,Date of Birth,3/4/2019,January


In [70]:
customers['Joining Date'] = customers['Joining Day'].astype('str') + ' ' + customers['month'].astype('str') + ', 2023'

In [71]:
customers['Joining Date'] = customers['Joining Date'].apply(lambda x: dt.strptime(x, "%d %B, %Y"))

In [72]:
customers.head(9)

,ID,Joining Day,Demographic,Value,month,Joining Date
0,490910,3,Ethnicity,White,January,2023-01-03
1,490910,3,Date of Birth,5/23/1981,January,2023-01-03
2,490910,3,Account Type,Basic,January,2023-01-03
3,369221,18,Ethnicity,Black,January,2023-01-18
4,369221,18,Date of Birth,3/4/2019,January,2023-01-18
5,369221,18,Account Type,Platinum,January,2023-01-18
6,568991,16,Ethnicity,White,January,2023-01-16
7,568991,16,Date of Birth,4/27/1979,January,2023-01-16
8,568991,16,Account Type,Gold,January,2023-01-16


In [73]:
customers = customers.pivot_table(index=['ID', 'Joining Day', 'month', 'Joining Date'],
                          columns='Demographic',
                          values='Value',
                          aggfunc=lambda x: x.iloc[0])
customers.reset_index(inplace=True)
customers.columns.name = None

In [74]:
customers_sorted = customers.sort_values(by=['Joining Date'])
customers = customers_sorted.drop_duplicates(subset=['ID'], keep='first')

In [75]:
customers.head()

,ID,Joining Day,month,Joining Date,Account Type,Date of Birth,Ethnicity
871,893948,1,January,2023-01-01,Gold,9/1/2013,White
837,863243,1,January,2023-01-01,Platinum,12/30/2007,Other
847,871764,2,January,2023-01-02,Gold,10/29/1967,White
725,761244,2,January,2023-01-02,Platinum,12/26/1942,White
393,471554,2,January,2023-01-02,Basic,4/22/1990,White
